In [ ]:
import requests, re, os, time, pickle
import pandas as pd

In [ ]:
keywords = ''''''
APIkey = ''

In [ ]:
extractedData =pd.DataFrame()
page = 1
count =25

In [ ]:

while True:
    url = 'https://api.elsevier.com/content/search/scopus?start={}&count={}&query={}'.format(page,count,keywords)
    resp = requests.get(url,headers={'Accept':'application/json','X-ELS-APIKey':APIkey})
    results = resp.json()
    if 'service-error' in results:
        break
        
    
    for article in results['search-results']['entry']:
        apk=None
        abst=None
        identifier=article['dc:identifier']


        basicInfoURL=article['prism:url']
        results_info = requests.get(basicInfoURL,headers={'Accept':'application/json','X-ELS-APIKey':APIkey})
        results_info = results_info.json()
        coredata = results_info['abstracts-retrieval-response']['coredata']
        authors=[author['ce:indexed-name'] for author in coredata['dc:creator']['author']]
        citation=coredata['citedby-count']

        fullInfoURL="https://api.elsevier.com/content/article/"+identifier
        resp_info = requests.get(fullInfoURL,headers={'Accept':'application/json','X-ELS-APIKey': APIkey})
        results_info = resp_info.json()
        
        if 'full-text-retrieval-response' in results_info:
            coredata = results_info["full-text-retrieval-response"]["coredata"]
            apk=[kw['$'] for kw in coredata['dcterms:subject']]
            abst=coredata['dc:description']
            authors=[author['$'] for author in coredata['dc:creator']]
                
        keyID=coredata['dc:identifier']
        title=coredata['dc:title']
        publishDate=coredata['prism:coverDate']
        location=coredata['prism:publicationName']
        articleType=coredata['prism:aggregationType']
        

        extractedData=extractedData.append({'keyID':keyID,'title':title,'date':publishDate,'authors':authors,
                        'keywords':apk,'journal_or_book_name':location,
                        'article_type':articleType,'abstract':abst,"citation_count":int(citation)},ignore_index=True)

    
    pickle.dump(extractedData,open('extractedData.p', "wb" ))
    print('{} articles are extracted | page {}'.format(extractedData.shape[0],page))
    page+=1

extractedData=extractedData[['keyID','title','date','authors','keywords','journal_or_book_name','article_type','citation_count','abstract']]

In [ ]:
extractedData.head()